# Import dependencies

In [1]:
import pandas as pd
import numpy as np
import json
import io
import datetime as dt
import string
import unicodedata

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer = ToktokTokenizer()
stopword_list = nltk.corpus.stopwords.words('english')

import spacy
nlp = spacy.load('en_core', parse=True, tag=True, entity=True)

import re
from bs4 import BeautifulSoup


from gensim import corpora, models, similarities

#nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)



    Only loading the 'en' tokenizer.



# Read in data

In [48]:
#create dataframe from input data; identify the column with the tweet text
df=pd.read_csv('Tweets.csv')
print('Number of observations are: '+str(len(df)))

Number of observations are: 14640


In [49]:
#remove any rows that has no tweet text
df=df.text.dropna()
df = df.reset_index(drop=True)
print('Number of observations are: '+str(len(df)))

Number of observations are: 14640


In [59]:
#Create dictionary of all tweets
tweet_dictionary = {}
i = 0
for line in df:
        tweet_dictionary[i] = line.lower()
        i += 1
print(tweet_dictionary[1])

@virginamerica plus you've added commercials to the experience... tacky.


# Data Preprocessing I

Remove HTML Links

In [60]:
def strip_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

In [61]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_links(tweet_dictionary[i])

Remove mentions

In [62]:
def strip_mentions(text):
    entity_prefixes = ['@']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [63]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_mentions(tweet_dictionary[i])

Remove hashtags

In [11]:
def strip_hashtags(text):
    entity_prefixes = ['#']
    for separator in  string.punctuation:
        if separator not in entity_prefixes :
            text = text.replace(separator,' ')
    words = []
    for word in text.split():
        word = word.strip()
        if word:
            if word[0] not in entity_prefixes:
                words.append(word)
    return ' '.join(words)

In [12]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=strip_hashtags(tweet_dictionary[i])

Remove retweet (RT) designation

In [64]:
for i in range(0,len(df)):
    tweet_dictionary[i] = tweet_dictionary[i].replace('RT', '')

# Data Preprocessing II

Expand contractions

In [65]:
#code source is http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

import re
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "youyou will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    text = c_re.sub(replace, text.lower())
    return text

In [66]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=expandContractions(tweet_dictionary[i])


Remove spcial or accented characters (and maybe numbers)

In [67]:
def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-z0-9\s]' if not remove_digits else r'[^a-zA-z\s]'
    text = re.sub(pattern, '', text)
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

In [68]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_special_characters(tweet_dictionary[i], 
                          remove_digits=True)

Stopwords

In [69]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stopword_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text

In [70]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=remove_stopwords(tweet_dictionary[i])

Stemming / Lemming

In [18]:
def simple_stemmer(text):
    ps = nltk.porter.PorterStemmer()
    text = ' '.join([ps.stem(word) for word in text.split()])
    return text

In [62]:
def lemmatize_text(text):
    text = nlp(text)
    text = ' '.join([word.lemma_ if word.lemma_ != '-PRON-' else word.text for word in text])
    return text

In [19]:
for i in range(0,len(tweet_dictionary)):
    tweet_dictionary[i]=simple_stemmer(tweet_dictionary[i])
    #tweet_dictionary[i]=lemmatize_text(tweet_dictionary[i])

Tokenization

In [71]:
#Create corupus of all words
words_corpus = []
for i in range(0,len(tweet_dictionary)):
        words_corpus.append([word for word in tweet_dictionary[i].lower().split() if len(word)>3])
print(words_corpus)

[['said'], ['plus', 'added', 'commercials', 'experience', 'tacky'], ['today', 'must', 'mean', 'need', 'take', 'another', 'trip'], ['really', 'aggressive', 'blast', 'obnoxious', 'entertainment', 'guests', 'faces', 'little', 'recourse'], ['really', 'thing'], ['seriously', 'would', 'flight', 'seats', 'playing', 'really', 'thing', 'flying'], ['nearly', 'every', 'time', 'worm', 'wont', 'away'], ['really', 'missed', 'prime', 'opportunity', 'without', 'hats', 'parody'], ['well', 'tbut'], ['amazing', 'arrived', 'hour', 'early', 'good'], ['know', 'suicide', 'second', 'leading', 'cause', 'death', 'among', 'teens'], ['pretty', 'graphics', 'much', 'better', 'minimal', 'iconography'], ['great', 'deal', 'already', 'thinking', 'trip', 'even', 'gone', 'trip'], ['flying', 'fabulous', 'seductive', 'skies', 'take', 'stress', 'away', 'travel'], ['thanks'], ['schedule', 'still'], ['excited', 'first', 'cross', 'country', 'flight', 'heard', 'nothing', 'great', 'things', 'virgin', 'america', 'daystogo'], ['fl

In [72]:
dictionary = corpora.Dictionary(words_corpus)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
# no_below is minimum number of documents
# no_above is a percentage
# keep_n is the thresshold for the max number of words to keep
print(dictionary)

Dictionary(1082 unique tokens: ['said', 'added', 'experience', 'plus', 'another']...)


Bag of Words 

In [73]:
corpus_bow = [dictionary.doc2bow(text) for text in words_corpus]

Term Frequency - Inverse Document Frequency

In [74]:
from gensim import corpora, models
tfidf = models.TfidfModel(corpus_bow)
corpus_tfidf = tfidf[corpus_bow]

# Data Analysis

Part of Speech Tagging

In [23]:
sentence = 'London is the capital and most populous city of England and the United Kingdom'

In [24]:
sentence_nlp = nlp(sentence)

# POS tagging with Spacy 
spacy_pos_tagged = [(word, word.tag_, word.pos_) for word in sentence_nlp]
pd.DataFrame(spacy_pos_tagged, columns=['Word', 'POS tag', 'Tag type'])

# POS tagging with nltk
#nltk_pos_tagged = nltk.pos_tag(sentence.split())
#pd.DataFrame(nltk_pos_tagged, columns=['Word', 'POS tag'])

,Word,POS tag,Tag type
0,London,NNP,PROPN
1,is,VBZ,VERB
2,the,DT,DET
3,capital,NN,NOUN
4,and,CC,CCONJ
5,most,RBS,ADV
6,populous,JJ,ADJ
7,city,NN,NOUN
8,of,IN,ADP
9,England,NNP,PROPN


Dependency Graph

In [27]:
from spacy import displacy
displacy.render(sentence_nlp, jupyter=True, 
                options={'distance': 110,
                         'arrow_stroke': 2,
                         'arrow_width': 8})

Named Entity Recognition

In [26]:
# print named entities in article
print([(word, word.ent_type_) for word in sentence_nlp if word.ent_type_])

# visualize named entities
displacy.render(sentence_nlp, style='ent', jupyter=True)

[(London, 'GPE'), (England, 'GPE'), (the, 'GPE'), (United, 'GPE'), (Kingdom, 'GPE')]


Similarity

In [104]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)
query = sentence
vec_bow = dictionary.doc2bow(query.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)
index = similarities.MatrixSimilarity(lsi[corpus])
sims = index[vec_lsi]

[(0, 0.0004018640463104192), (1, -5.492274233986752e-05)]


# Topic Modeling

Topic modeling using Bag of Words

In [75]:
lda_model = models.LdaMulticore(corpus_bow, num_topics=10, id2word=dictionary, passes=2, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.061*"thank" + 0.024*"sent" + 0.018*"response" + 0.015*"plane" + 0.015*"hold" + 0.014*"would" + 0.014*"going" + 0.013*"really" + 0.013*"care" + 0.012*"help"
Topic: 1 
Words: 0.109*"flight" + 0.056*"cancelled" + 0.047*"flightled" + 0.023*"help" + 0.017*"today" + 0.017*"flights" + 0.012*"thanks" + 0.011*"back" + 0.011*"already" + 0.010*"call"
Topic: 2 
Words: 0.036*"like" + 0.025*"jetblue" + 0.016*"fleek" + 0.015*"fleet" + 0.013*"right" + 0.013*"phone" + 0.012*"home" + 0.012*"trip" + 0.011*"still" + 0.009*"upgrade"
Topic: 3 
Words: 0.049*"service" + 0.039*"customer" + 0.033*"thanks" + 0.021*"flight" + 0.019*"guys" + 0.019*"much" + 0.013*"thank" + 0.013*"good" + 0.013*"seat" + 0.012*"made"
Topic: 4 
Words: 0.022*"flight" + 0.019*"would" + 0.014*"still" + 0.013*"take" + 0.012*"thanks" + 0.012*"please" + 0.011*"hope" + 0.010*"ticket" + 0.009*"time" + 0.009*"love"
Topic: 5 
Words: 0.057*"flight" + 0.041*"thanks" + 0.041*"cancelled" + 0.023*"weather" + 0.021*"flighted" + 0.0

In [76]:
for index, score in sorted(lda_model[corpus_bow[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.7749595046043396	 
Topic: 0.036*"like" + 0.025*"jetblue" + 0.016*"fleek" + 0.015*"fleet" + 0.013*"right" + 0.013*"phone" + 0.012*"home" + 0.012*"trip" + 0.011*"still" + 0.009*"upgrade"

Score: 0.025011472404003143	 
Topic: 0.049*"service" + 0.039*"customer" + 0.033*"thanks" + 0.021*"flight" + 0.019*"guys" + 0.019*"much" + 0.013*"thank" + 0.013*"good" + 0.013*"seat" + 0.012*"made"

Score: 0.025008032098412514	 
Topic: 0.057*"flight" + 0.041*"thanks" + 0.041*"cancelled" + 0.023*"weather" + 0.021*"flighted" + 0.013*"want" + 0.012*"airport" + 0.011*"trying" + 0.011*"flights" + 0.010*"airline"

Score: 0.025004878640174866	 
Topic: 0.037*"time" + 0.034*"gate" + 0.031*"flight" + 0.022*"service" + 0.019*"first" + 0.019*"plane" + 0.016*"customer" + 0.012*"wait" + 0.012*"next" + 0.010*"flights"

Score: 0.02500457689166069	 
Topic: 0.034*"call" + 0.032*"hold" + 0.029*"need" + 0.028*"help" + 0.025*"please" + 0.022*"flight" + 0.018*"phone" + 0.016*"back" + 0.015*"hours" + 0.014*"trying"



Topic modeling using TF-IDF

In [77]:
lda_model_tfidf = models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.012*"service" + 0.012*"guys" + 0.012*"flight" + 0.010*"customer" + 0.009*"help" + 0.008*"next" + 0.008*"love" + 0.008*"would" + 0.007*"suck" + 0.007*"change"
Topic: 1 Word: 0.015*"flight" + 0.010*"tomorrow" + 0.010*"phone" + 0.010*"number" + 0.009*"plane" + 0.008*"minutes" + 0.008*"time" + 0.008*"hours" + 0.007*"hour" + 0.007*"hold"
Topic: 2 Word: 0.017*"flight" + 0.008*"make" + 0.008*"year" + 0.007*"cancelled" + 0.007*"going" + 0.007*"book" + 0.007*"plane" + 0.007*"thanks" + 0.006*"travel" + 0.006*"want"
Topic: 3 Word: 0.017*"flight" + 0.016*"need" + 0.013*"delayed" + 0.013*"response" + 0.012*"late" + 0.011*"cancelled" + 0.008*"help" + 0.007*"work" + 0.007*"flightled" + 0.007*"flightr"
Topic: 4 Word: 0.045*"thanks" + 0.019*"service" + 0.015*"customer" + 0.012*"flight" + 0.010*"airline" + 0.010*"call" + 0.009*"great" + 0.009*"time" + 0.008*"flights" + 0.008*"never"
Topic: 5 Word: 0.038*"thank" + 0.017*"fleek" + 0.016*"fleet" + 0.014*"flight" + 0.012*"jetblue" + 0.009*"

In [78]:
for index, score in sorted(lda_model_tfidf[corpus_bow[1]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.7749748826026917	 
Topic: 0.038*"thank" + 0.017*"fleek" + 0.016*"fleet" + 0.014*"flight" + 0.012*"jetblue" + 0.009*"weather" + 0.008*"flights" + 0.008*"passengers" + 0.006*"much" + 0.006*"give"

Score: 0.02500610053539276	 
Topic: 0.012*"service" + 0.012*"guys" + 0.012*"flight" + 0.010*"customer" + 0.009*"help" + 0.008*"next" + 0.008*"love" + 0.008*"would" + 0.007*"suck" + 0.007*"change"

Score: 0.025003856047987938	 
Topic: 0.021*"flight" + 0.015*"thank" + 0.012*"cancelled" + 0.011*"service" + 0.010*"customer" + 0.010*"great" + 0.009*"flightled" + 0.009*"plane" + 0.009*"thanks" + 0.008*"know"

Score: 0.025003179907798767	 
Topic: 0.016*"hold" + 0.014*"gate" + 0.013*"please" + 0.011*"flight" + 0.011*"like" + 0.010*"back" + 0.010*"hours" + 0.008*"time" + 0.008*"help" + 0.008*"going"

Score: 0.025002894923090935	 
Topic: 0.017*"flight" + 0.008*"make" + 0.008*"year" + 0.007*"cancelled" + 0.007*"going" + 0.007*"book" + 0.007*"plane" + 0.007*"thanks" + 0.006*"travel" + 0.006*"want

# Sentiment

In [105]:
#find sentiment vader
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [106]:
snt = analyser.polarity_scores('This is an examle of a happy tweet')
print(snt['compound'])
      

0.5719
